In [2]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import json
import pandas as pd

# Activate Driver

In [3]:
# Start a new Chrome browser
driver = webdriver.Chrome()

# Get All Problem URLs


In [ ]:
# Initialize
links = []

# Open zeroJudge problems page
driver.get("https://zerojudge.tw/Problems")


while True:
    try:
        # Get problem titles
        titleElements = driver.find_elements(
            By.CSS_SELECTOR,
            "body > div.container > div > table > tbody > tr > td:nth-child(3) > a",
        )

        # Get problem links
        for titleElement in titleElements:
            links.append(titleElement.get_attribute("href"))

        # Find next page link (button)
        nextLinkElement = None
        nextPageElements = driver.find_elements(By.CSS_SELECTOR, "#pagging > a")
        for nextPageElement in nextPageElements:
            if nextPageElement.text == "下一頁":
                nextLinkElement = nextPageElement
                break
        if nextLinkElement is None:
            break
        nextLinkElement.click()
    except NoSuchElementException:
        break

# Export Links
with open("ProblemLinks.json", "w") as file:
    json.dump(links, file)

# Scrape Data From Problem URLs


In [74]:
# Initialize
probNums = []
titles = []
tags = []
passRates = []
refs = []

# Load Links
links = []
with open("ProblemLinks.json", "r") as file:
    links = json.load(file)

# Scrape Data
for link in links:
    # link = links[0]
    driver.get(link)

    # probNums
    probNum = link.split("=")[-1]

    # titles
    titleElement = driver.find_element(By.ID, "problem_title")
    title = titleElement.text

    # tags
    try:
        tagElement = driver.find_element(
            By.CSS_SELECTOR,
            "body > div.container > div.row.Consolas > div.col-md-4 > span.tag",
        )
        tag = tagElement.text
    except NoSuchElementException:
        tag = ""

    # passRates
    try:
        passRateElement = driver.find_element(
            By.CSS_SELECTOR,
            "span[title='解題統計']",
        )
        passRateText = passRateElement.text
        passRate = passRateText[
            passRateText.index("(") + 1 : passRateText.index(")")
        ].strip()
    except NoSuchElementException:
        passRate = ""

    # refs
    try:
        refElement = driver.find_element(
            By.CSS_SELECTOR,
            "body > div.container > div:nth-child(2) > div > div:nth-child(5) > div",
        )
        ref = refElement.text
    except NoSuchElementException:
        ref = ""

    # Append Data
    probNums.append(probNum)
    titles.append(title)
    tags.append(tag)
    passRates.append(passRate)
    refs.append(ref)

# Export to Excel
data = {
    "題號": probNums,
    "題目": titles,
    "標籤": tags,
    "通過率": passRates,
    "出處": refs,
}

# Import pandas as pd
pd.DataFrame(data).to_excel("Zerojudge.xlsx", index=False)

# Quit

In [4]:
# Close the browser
driver.quit()